#### Create word embedding from corpus by Pretrained News vectors

In [1]:
from __future__ import unicode_literals, print_function
import gensim, logging, os
from gensim.models import Word2Vec, Phrases, phrases, KeyedVectors
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import tokenize
import string
from itertools import chain

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
#word2vec.FAST_VERSION

In [2]:
content_dir = '../../Data/input/train/content'

class Corpus_Sentences(object):
    def __init__(self, dirname):
        self.dirname = dirname;
        
    def __iter__(self):
        for fname in os.listdir(self.dirname):
            for line in open(os.path.join(self.dirname, fname)):
                yield line.split()
                
sentences = Corpus_Sentences(content_dir) # a memory-friendly iterator

In [3]:
# Load the pretrained glove vectors
glove_wv = KeyedVectors.load_word2vec_format('../../../word_embeddings/pretrained_word_embeddings/Word2Vec/Glove_Converted/glove_2_word2vec.6B.300d.txt', binary=False)

2018-05-24 14:33:48,952 : INFO : loading projection weights from ../../../word_embeddings/pretrained_word_embeddings/Word2Vec/Glove_Converted/glove_2_word2vec.6B.300d.txt
2018-05-24 14:35:48,713 : INFO : loaded (400000, 300) matrix from ../../../word_embeddings/pretrained_word_embeddings/Word2Vec/Glove_Converted/glove_2_word2vec.6B.300d.txt


In [4]:
# Build vocabulary from corpus and words from pretrauned vector

model = Word2Vec(size=300, min_count=1, iter=10)
model.build_vocab(sentences)
training_examples_count = model.corpus_count
# below line will make it 1, so saving it before
model.build_vocab([list(glove_wv.vocab.keys())], update=True)

2018-05-24 14:35:52,072 : INFO : collecting all words and their counts
2018-05-24 14:35:52,098 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-05-24 14:35:57,837 : INFO : collected 17349 word types from a corpus of 175106 raw words and 895 sentences
2018-05-24 14:35:57,838 : INFO : Loading a fresh vocabulary
2018-05-24 14:35:57,948 : INFO : min_count=1 retains 17349 unique words (100% of original 17349, drops 0)
2018-05-24 14:35:57,949 : INFO : min_count=1 leaves 175106 word corpus (100% of original 175106, drops 0)
2018-05-24 14:35:58,074 : INFO : deleting the raw counts dictionary of 17349 items
2018-05-24 14:35:58,075 : INFO : sample=0.001 downsamples 23 most-common words
2018-05-24 14:35:58,076 : INFO : downsampling leaves estimated 166054 word corpus (94.8% of prior 175106)
2018-05-24 14:35:58,164 : INFO : estimated required memory for 17349 words and 300 dimensions: 50312100 bytes
2018-05-24 14:35:58,166 : INFO : resetting layer weights
2018-05-24 

In [5]:
#if lockf=0.0 then pretrained word vectors will not be trained so keeping lockf=1.0
# Update the weights from pretrained vectors

model.intersect_word2vec_format("../../../word_embeddings/pretrained_word_embeddings/Word2Vec/Glove_Converted/glove_2_word2vec.6B.300d.txt",binary=False, lockf=1.0)

2018-05-24 14:36:57,572 : INFO : loading projection weights from ../../../word_embeddings/pretrained_word_embeddings/Word2Vec/Glove_Converted/glove_2_word2vec.6B.300d.txt
2018-05-24 14:39:02,380 : INFO : merged 400000 vectors into (403308, 300) matrix from ../../../word_embeddings/pretrained_word_embeddings/Word2Vec/Glove_Converted/glove_2_word2vec.6B.300d.txt


In [6]:
# Train the model
model.train(sentences, total_examples=training_examples_count, epochs=100)

2018-05-24 14:39:09,664 : INFO : training model with 3 workers on 403308 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2018-05-24 14:39:10,883 : INFO : EPOCH 1 - PROGRESS: at 7.93% examples, 9294 words/s, in_qsize 0, out_qsize 0
2018-05-24 14:39:12,005 : INFO : EPOCH 1 - PROGRESS: at 25.14% examples, 13602 words/s, in_qsize 0, out_qsize 0
2018-05-24 14:39:13,266 : INFO : EPOCH 1 - PROGRESS: at 39.22% examples, 14394 words/s, in_qsize 0, out_qsize 0
2018-05-24 14:39:14,313 : INFO : EPOCH 1 - PROGRESS: at 54.75% examples, 17552 words/s, in_qsize 0, out_qsize 0
2018-05-24 14:39:15,644 : INFO : EPOCH 1 - PROGRESS: at 69.94% examples, 18567 words/s, in_qsize 0, out_qsize 0
2018-05-24 14:39:16,817 : INFO : EPOCH 1 - PROGRESS: at 84.36% examples, 19669 words/s, in_qsize 0, out_qsize 0
2018-05-24 14:39:18,079 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 14:39:18,083 : INFO : EPOCH 1 - PROGRESS: at 96.54% examples, 19994 words/s, 

2018-05-24 14:39:30,983 : INFO : EPOCH - 18 : training on 175106 raw words (175106 effective words) took 0.7s, 254531 effective words/s
2018-05-24 14:39:31,763 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 14:39:31,797 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-24 14:39:31,820 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-24 14:39:31,824 : INFO : EPOCH - 19 : training on 175106 raw words (175106 effective words) took 0.8s, 208823 effective words/s
2018-05-24 14:39:32,450 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 14:39:32,484 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-24 14:39:32,491 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-24 14:39:32,492 : INFO : EPOCH - 20 : training on 175106 raw words (175106 effective words) took 0.7s, 263080 effective words/s
2018-05-24 14:39:33,150 : INFO : worker thread

2018-05-24 14:39:46,062 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 14:39:46,104 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-24 14:39:46,128 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-24 14:39:46,129 : INFO : EPOCH - 39 : training on 175106 raw words (175106 effective words) took 0.8s, 207473 effective words/s
2018-05-24 14:39:46,976 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 14:39:47,044 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-24 14:39:47,059 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-24 14:39:47,060 : INFO : EPOCH - 40 : training on 175106 raw words (175106 effective words) took 0.9s, 189044 effective words/s
2018-05-24 14:39:48,109 : INFO : EPOCH 41 - PROGRESS: at 79.89% examples, 122892 words/s, in_qsize 0, out_qsize 0
2018-05-24 14:39:48,241 : INFO : worker thread finished; awaiting fi

2018-05-24 14:39:59,590 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 14:39:59,625 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-24 14:39:59,631 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-24 14:39:59,632 : INFO : EPOCH - 59 : training on 175106 raw words (175106 effective words) took 0.6s, 291090 effective words/s
2018-05-24 14:40:00,241 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 14:40:00,271 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-24 14:40:00,284 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-24 14:40:00,285 : INFO : EPOCH - 60 : training on 175106 raw words (175106 effective words) took 0.7s, 269325 effective words/s
2018-05-24 14:40:00,940 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 14:40:00,981 : INFO : worker thread finished; awaiting finish of 1 more threads


2018-05-24 14:40:16,523 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 14:40:16,561 : INFO : EPOCH 78 - PROGRESS: at 96.54% examples, 164951 words/s, in_qsize 1, out_qsize 1
2018-05-24 14:40:16,563 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-24 14:40:16,595 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-24 14:40:16,596 : INFO : EPOCH - 78 : training on 175106 raw words (175106 effective words) took 1.0s, 168905 effective words/s
2018-05-24 14:40:17,186 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 14:40:17,204 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-24 14:40:17,230 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-24 14:40:17,231 : INFO : EPOCH - 79 : training on 175106 raw words (175106 effective words) took 0.6s, 277578 effective words/s
2018-05-24 14:40:17,786 : INFO : worker thread finished; awaiting fi

2018-05-24 14:40:29,786 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 14:40:29,804 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-24 14:40:29,830 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-24 14:40:29,831 : INFO : EPOCH - 98 : training on 175106 raw words (175106 effective words) took 0.6s, 269808 effective words/s
2018-05-24 14:40:30,406 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 14:40:30,437 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-05-24 14:40:30,449 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-05-24 14:40:30,449 : INFO : EPOCH - 99 : training on 175106 raw words (175106 effective words) took 0.6s, 284029 effective words/s
2018-05-24 14:40:31,007 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-05-24 14:40:31,037 : INFO : worker thread finished; awaiting finish of 1 more threads


(17510600, 17510600)

In [7]:
#model.wv.save_word2vec_format('../../trained_word_embeddings/word2vec/word_pretrain_trained_on_corpus/w2v_pretain_corpus_trained_gensim_300.bin')
model.wv.save_word2vec_format('../../trained_word_embeddings/word2vec/word_pretrain_trained_on_corpus/w2v_pretain_corpus_trained_gensim_300.txt', binary=False)

2018-05-24 14:46:15,201 : INFO : storing 403308x300 projection weights into ../../trained_word_embeddings/word2vec/word_pretrain_trained_on_corpus/w2v_pretain_corpus_trained_gensim_300.txt
